In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as  np
batch_size=64
epochs=100
latent_dim=256 #neurons in the LSTM
num_samples=2500
data_path='hin.txt'

In [6]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines= f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text,target_text,_=line.split('\t')
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text :
        if char not in target_characters:
            target_characters.add(char)


In [14]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [15]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens) #vocabulary size of english
print('Number of unique output tokens:', num_decoder_tokens) #vocabulary size of hindi
print('Max sequence length for inputs:', max_encoder_seq_length) #max length in input word as english
print('Max sequence length for outputs:', max_decoder_seq_length) #max length of output word as hindi

Number of samples: 2500
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 40
Max sequence length for outputs: 67


In [19]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict(
    [(char,i) for i,char in enumerate(target_characters)])

In [22]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [26]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i,t+1:, input_token_index[' '] = 1.
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1.
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    decoder_input_data[i, t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']]=1

In [ ]:
# this stores the one-hot encoded input data (at the character level), where:
# i → index of the sample (sentence)
# t → current character index
# input_token_index[' '] → index of the space character ' ' in the vocabulary

In [ ]:
# encoder_inputs: shape (batch_size, timesteps, num_encoder_tokens)
# encoder_input_data: shape (num_samples, max_encoder_seq_length, num_encoder_tokens)

In [ ]:
# hidden state (ℎ)	The output at each time step. It’s also called the short-term memory.
# cell state (𝑐)	The internal memory of the cell — stores long-term information.

In [27]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [29]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 628ms/step - accuracy: 0.5863 - loss: 2.5145 - val_accuracy: 0.5415 - val_loss: 2.2267
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 633ms/step - accuracy: 0.6719 - loss: 1.6464 - val_accuracy: 0.5460 - val_loss: 2.0162
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 658ms/step - accuracy: 0.6735 - loss: 1.4711 - val_accuracy: 0.5460 - val_loss: 2.3078
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 621ms/step - accuracy: 0.6731 - loss: 1.4308 - val_accuracy: 0.5460 - val_loss: 2.0926
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 605ms/step - accuracy: 0.6778 - loss: 1.3877 - val_accuracy: 0.5460 - val_loss: 1.9538
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 654ms/step - accuracy: 0.6753 - loss: 1.3550 - val_accuracy: 0.5461 - val_loss: 1.8681
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 646ms/step - accuracy: 0.6747 - loss: 1.3255 - val_accuracy: 0.5126 - val_loss: 1.8862
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 607ms/step - accuracy: 0.6798 - loss: 1.2942 - 

In [33]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)


In [34]:
# Decoder setup for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [35]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [36]:
def decode_sequence(input_seq):
    # The encoder LSTM takes the input and outputs its final hidden and cell states (states_value).
    # These states represent the context of the input sentence.
    # They’ll be used to initialize the decoder.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence with only the start character.
    # The decoder needs a **starting point** — this tells it: *"Start generating now!"*
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Output sequence
    stop_condition = False
    decoded_sentence = ''
    # A loop for step-by-step prediction.**  
    # - The decoder predicts **one character at a time**.
    # - We keep feeding the previous output as the next input.
    while not stop_condition:
        target_seq: the last generated character

     states_value: the last hidden & cell states
        # output_tokens: the probabilities for next character
        # h, c: the updated LSTM states
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # - Find the **most likely next character** (`argmax`).
        # - Convert its index back to the **actual character** (`reverse_target_char_index`).
        # - Add it to your output sentence.
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: end character or max length
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [39]:
# Encode your input string like during training
input_text = "how are you"
encoder_input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for t, char in enumerate(input_text):
    if char in input_token_index:
        encoder_input_seq[0, t, input_token_index[char]] = 1.
    else:
        print(f"Warning: '{char}' not in input token index!")

# Predict
decoded_sentence = decode_sequence(encoder_input_seq)
print('Input:', input_text)
print('Decoded:', decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 